In [1]:
import pandas as pd
import numpy as np
import os
from sklearn import preprocessing
from sklearn import confusion_matrix
from sklearn.model_selection import StratifiedKFold,KFold,ShuffleSplit
import cntk as c
import matplotlib.pyplot as plt
from keras import backend as c
from keras import layers
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.optimizers import SGD, Adam
from keras import regularizers

C:\Users\Devanshu\Anaconda3\envs\cntk_env\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using CNTK backend
C:\Users\Devanshu\Anaconda3\envs\cntk_env\lib\site-packages\keras\backend\cntk_backend.py:21: UserWarning: CNTK backend warning: GPU is not detected. CNTK's CPU version is not fully optimized,please run with GPU to get better performance.
  'CNTK backend warning: GPU is not detected. '


In [2]:
x_train= np.loadtxt('C:/Users/Devanshu/Desktop/faltu/data without misslabel/train_test_without_miss_normalize.csv', delimiter=',', dtype='float32')
x_train_label= np.loadtxt('C:/Users/Devanshu/Desktop/faltu/data without misslabel/label_encoded_train.csv', delimiter=',', dtype='float32')
x_test_label= np.loadtxt('C:/Users/Devanshu/Desktop/faltu/data without misslabel/label_encoded_test.csv', delimiter=',', dtype='float32')
print(x_train.shape)
print(x_train_label.shape)
print(x_test_label.shape)

(144766, 40)
(125973, 5)
(18793, 5)


In [3]:
train= x_train[0:125973,:]
test= x_train[125973:,:]
train_label= x_train_label
test_label= x_test_label
print(train.shape, train_label.shape,test.shape, test_label.shape)
#print(train.shape, train_label.shape,test.shape)

(125973, 40) (125973, 5) (18793, 40) (18793, 5)


In [4]:
train= train.reshape(125973,5,8)
test= test.reshape(18793,5,8)
input_dimension= 40

In [7]:
model1= Sequential()

model1.add(Conv1D(strides=1, input_shape=(5, 8), padding="same", activation="tanh", filters=32, kernel_size=2, name="layer1"))

ly= model1.layers[0].get_weights()
    
model1.add(Flatten())

model1.add(Dense(128, input_dim=input_dimension, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model1.add(Dropout(0.2))
model1.add(Dense(5, activation='softmax'))

adam=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model1.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['acc'])

print(model1.summary())

model1.fit(train, train_label, epochs=4, batch_size=None,validation_split=0.2)

#model1.save_weights('C:/Users/Devanshu/Desktop/faltu/model_saved/weights1.hdf5')

scores= model1.evaluate(test,test_label)
print(scores[1]*100)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Conv1D)              (None, 5, 32)             544       
_________________________________________________________________
flatten_3 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               20608     
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 645       
Total params: 21,797
Trainable params: 21,797
Non-trainable params: 0
_________________________________________________________________
None
Train on 100778 samples, validate on 25195 samples
Epoch 1/4
100778/100778 [==============================] - 39s 382us/step - loss: 

In [8]:
model2= Sequential()


m1= model2.add(Conv1D(strides=1, input_shape=(5, 8), padding="same", activation="tanh", filters=32, kernel_size=2, name="layer1"))
model2.layers[0].set_weights(ly)

model2.add(MaxPooling1D(pool_size=1, strides=None, padding='same'))

model2.add(Conv1D(strides=1, padding="same", activation="tanh", filters=16, kernel_size=1, name="layer2"))
ly2= model2.layers[2].get_weights()
    
model2.add(Flatten())

model2.add(Dense(128, input_dim=input_dimension, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model2.add(Dropout(0.2))
model2.add(Dense(5, activation='softmax'))

adam=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model2.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['acc'])

print(model2.summary())

model2.fit(train, train_label, epochs=4, batch_size=None, validation_split=0.2)

#model1.save_weights('C:/Users/Devanshu/Desktop/faltu/model_saved/weights1.hdf5')

scores= model2.evaluate(test,test_label)
print(scores[1]*100)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Conv1D)              (None, 5, 32)             544       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 5, 32)             0         
_________________________________________________________________
layer2 (Conv1D)              (None, 5, 16)             528       
_________________________________________________________________
flatten_4 (Flatten)          (None, 80)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               10368     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 5)                 645       
Total para

In [9]:
model3= Sequential()


m1= model3.add(Conv1D(strides=1, input_shape=(5, 8), padding="same", activation="tanh", filters=32, kernel_size=2, name="layer1"))
model3.layers[0].set_weights(ly)

model3.add(MaxPooling1D(pool_size=1, strides=None, padding='same'))

model3.add(Conv1D(strides=1, padding="same", activation="tanh", filters=16, kernel_size=1, name="layer2"))
model3.layers[2].set_weights(ly2)
    
model3.add(Dropout(0.2))

model3.add(Conv1D(strides=1, padding="same", activation="tanh", filters=8, kernel_size=1,name="layer3"))
ly3= model3.layers[4].get_weights()

model3.add(MaxPooling1D(pool_size=1, strides=None, padding='same'))
    
model3.add(Flatten())

model3.add(Dense(128, input_dim=input_dimension, activation='relu',kernel_regularizer=regularizers.l2(0.01)))
model3.add(Dropout(0.2))
model3.add(Dense(5, activation='softmax'))

adam=Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model3.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['acc'])

print(model3.summary())

model3.fit(train, train_label, epochs=2, batch_size=None, verbose=1, validation_split=0.2)

#model1.save_weights('C:/Users/Devanshu/Desktop/faltu/model_saved/weights1.hdf5')

scores= model3.evaluate(test,test_label)
print(scores[1]*100)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
layer1 (Conv1D)              (None, 5, 32)             544       
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 5, 32)             0         
_________________________________________________________________
layer2 (Conv1D)              (None, 5, 16)             528       
_________________________________________________________________
dropout_5 (Dropout)          (None, 5, 16)             0         
_________________________________________________________________
layer3 (Conv1D)              (None, 5, 8)              136       
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 5, 8)              0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 40)                0         
__________

In [11]:
model2.predict(test)

array([[9.93846118e-01, 1.72786962e-03, 4.13719751e-03, 1.71926629e-04,
        1.16946736e-04],
       [9.79879558e-01, 7.07632070e-03, 1.23100253e-02, 4.71393345e-04,
        2.62731453e-04],
       [3.34519744e-02, 9.13971722e-01, 4.05859798e-02, 1.02462014e-02,
        1.74406567e-03],
       ...,
       [1.07356384e-02, 9.86730456e-01, 3.06433329e-04, 1.98092149e-03,
        2.46530137e-04],
       [1.38529930e-02, 9.83350694e-01, 3.30719078e-04, 2.18285434e-03,
        2.82797031e-04],
       [1.40383812e-02, 9.84393179e-01, 7.40309828e-04, 6.63856335e-04,
        1.64149635e-04]], dtype=float32)